In [49]:
'''CODE HERE'''
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
# import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
# from sklearn.linear_model import LogisticRegression
# #A random forest is a meta estimator that fits a number of decision tree classifiers 
# #on various sub-samples of the dataset and use averaging to improve the predictive 
# #accuracy and control over-fitting.
# from sklearn.ensemble import RandomForestClassifier
# #a discriminative classifier formally defined by a separating hyperplane.
# from sklearn.svm import SVC
# #displayd data
# from IPython.display import display
# %matplotlib inline


df = pd.read_csv('Data/finalData.csv')

In [51]:
# Preview data.
df.sort_values(by=['Date'])
df['Date'] = pd.to_datetime(df['Date'])

##returns the matches between start date and end date// yyyy-mm-dd
##display(getMatchesBeforGivenDate("2008-08-16","2010-08-17"))
def getMatchesBeforGivenDate(data,start_date,end_date):
    return data[data.Date.between(start_date, end_date)]




In [63]:
def getMatchesWithHomeTeam(data,teamName):
    return data.loc[data['HomeTeam'] == teamName]

def getMatchesWithAwayTeam(data,teamName):
    return data.loc[data['AwayTeam'] == teamName]

def getMatchesWithHomeTeamWin(data):
    return data.loc[data['FTR'] == "H"]

def getMatchesWithAwayTeamWin(data):
    return data.loc[data['FTR'] == "A"]

def getMatchesWithDraw(data):
    return data.loc[data['FTR'] == "D"]
    
def getHomeTeamWinPercent(teamName,toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df,"2005-01-01",toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange,teamName)
    homeTeamWinRows = getMatchesWithHomeTeamWin(teamRows)
    return len(homeTeamWinRows)/len(teamRows)

def getHomeTeamDrawPercent(teamName,toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df,"2005-01-01",toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange,teamName)
    homeTeamDrawRows = getMatchesWithDraw(teamRows)
    return len(homeTeamDrawRows)/len(teamRows)

def getAwayTeamWinPercent(teamName,toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df,"2005-01-01",toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange,teamName)
    awayTeamWinRows = getMatchesWithAwayTeamWin(teamRows)
    return len(awayTeamWinRows)/len(teamRows)

def getAwayTeamDrawPercent(teamName,toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df,"2005-01-01",toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange,teamName)
    awayTeamDrawRows = getMatchesWithDraw(teamRows)
    return len(awayTeamDrawRows)/len(teamRows)

def getAvgHomeTeamGoals(teamName,toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df,"2005-01-01",toDate)
    teamRows = getMatchesWithHomeTeam(matchesInDateRange,teamName)
    totalHomeGoals = teamRows['FTHG'].sum()
    display(teamRows)
    return totalHomeGoals/len(teamRows)

def getAvgAwayTeamGoals(teamName,toDate):
    matchesInDateRange = getMatchesBeforGivenDate(df,"2005-01-01",toDate)
    teamRows = getMatchesWithAwayTeam(matchesInDateRange,teamName)
    totalAwayGoals = teamRows['FTAG'].sum()
    display(teamRows)
    return totalAwayGoals/len(teamRows)

getAvgHomeTeamGoals("Arsenal","2008-09-30")

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,Unnamed: 71,Unnamed: 72,Unnamed: 73
0,E0,2008-08-16,Arsenal,West Brom,1,0,H,1,0,H,...,2.14,26.0,-1.50,1.88,1.81,2.12,1.97,NaN,NaN,NaN
20,E0,2008-08-30,Arsenal,Newcastle,3,0,H,2,0,H,...,1.82,21.0,-1.00,1.83,1.79,2.17,2.08,NaN,NaN,NaN
49,E0,2008-09-27,Arsenal,Hull,1,2,A,0,0,D,...,2.20,23.0,-1.75,1.89,1.84,2.12,2.03,NaN,NaN,NaN
109,E0,2008-08-11,Arsenal,Man United,2,1,H,1,0,H,...,1.71,19.0,0.25,1.98,1.90,2.01,1.96,NaN,NaN,NaN
149,E0,2008-06-12,Arsenal,Wigan,1,0,H,1,0,H,...,2.10,24.0,-1.50,2.09,2.03,1.86,1.81,NaN,NaN,NaN


1.6